В результате подготовительных операций был получен обучающий датасет изображений, который сохранён в папке **train**. Структура датанных разделена на подпапки **men** и **women**, каждая из которых в свою очередь имеет вложенные подпапки **eng** и **rus** для разделения знаменитостей на иностранных и отечественных. Для удобства подобной структуры планируется придерживаться во всём проекте.

Так как в идеале в готовом решении мы хотели бы получить сервис, который по одному загруженному изображению находит максимально похожих иностранных и отечественных знаменитостей, то первая проблема, которую предстоить решить - это автоматическое определение пола по загруженному изображению.

Для решения этой задачи необходимо обучить модель на определение пола по загруженному изображению. Для этого необходимо получить и сохранить для каждого изображения эмбединги на основе всё той же библиотеки *face_recognition* и сохранить их с сохранением структуры данных в папке **static**. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

from tqdm import notebook
import face_recognition
from PIL import Image
import numpy as np
import pickle
import glob
import os

### 1. Получение эмбедингов изображений

In [ ]:
# Функция получения и сохранения эмбедингов

def get_embedings(gender='men'):  
    pet_path = './Documents/DataScience/PET-project'
    
    # Создание папки для хранения эмбедингов
    if os.path.exists(f'{pet_path}/static') != True:
        os.mkdir(f'{pet_path}/static')
        
    # Создание подпапок для актеров и актрис
    if os.path.exists(f'{pet_path}/static/{gender}') != True:
        os.mkdir(f'{pet_path}/static/{gender}')
        
    nation = ['eng', 'rus']
    for natio in nation:
        # Создание подпапок для разделения знаменитостей на иностранных и отечественных
        if os.path.exists(f'{pet_path}/static/{gender}/{natio}') != True:
            os.mkdir(f'{pet_path}/static/{gender}/{natio}')
        
        # Получаем список знаменитостей из папки train
        folders = glob.glob(f'{pet_path}/train/{gender}/{natio}/*')
        
        dict_labels = dict()
        for i, name in enumerate(folders):
            if natio == 'eng':
                name = name[name.find('eng\\') + 4:].strip()
            elif natio == 'rus':
                name = name[name.find('rus\\') + 4:].strip()
            dict_labels[name] = i
        
        # Сохранение словаря с метками
        pkl_patch = f'{pet_path}/static/'
        
        if gender == 'men':
            role = 'actors'
        elif gender == 'women':
            role = 'actresses'
        
        with open(pkl_patch + f'{natio}-{role}-dict-labels.pkl','wb') as f:
            pickle.dump(dict_labels, f)
        
        # Создаём пустые массивы под эмбеддинги и метки
        embedings = np.empty(128)
        target = []
        
        for person in notebook.tqdm(list(dict_labels.keys())):
            images = os.listdir(f'{pet_path}/train/{gender}/{natio}/{person}')
            
            for num, person_img in enumerate(images):
                face = face_recognition.load_image_file(f'{pet_path}/train/{gender}/{natio}/{person}/{person_img}')
                try:
                    # Преобразуем фото с лицом в вектор, получаем embeding
                    face_enc = face_recognition.face_encodings(face)[0]
                    
                    # Добавляем в датасет матрицу
                    embedings = np.vstack((embedings, face_enc))
                    
                    # Добавляем таргет по текущему индексу
                    target.append(dict_labels[person])
                except Exception as ex:
                    print(f'Error message {ex}')
                    
            print(f'Embeding for :: {person} - successfully received !')
        
        # Удаляем из датасета первый элемент, так как это пустая матрица
        embedings_ = embedings[1:]
        
        with open(f'{pet_path}/static/{gender}/{natio}/' + 'embedings-train.pkl','wb') as f:
            pickle.dump(embedings_, f)
        
        with open(f'{pet_path}/static/{gender}/{natio}/' + 'labels-train.pkl','wb') as f:
            pickle.dump(target, f)

In [ ]:
get_embedings()

In [ ]:
get_embedings(gender='women')

### 2. Обучение модели для определения пола по загруженному изображению

После того, как получены эмбединги обучим модель для определения пола по загруженному изображению. В качестве бейзлайна попробуем использовать линенейную регрессию.

In [ ]:
# Функция для считывания сохраненных эмбедингов

def read_embedings(gender='men'):
    stat_path = './Documents/DataScience/PET-project/static'
    
    nation = ['eng', 'rus']
    target = []
    for natio in nation:
        with open(f'{stat_path}/{gender}/{natio}/embedings-train.pkl','rb') as f:
            if natio == 'eng':
                embedings = pickle.load(f)
            else:
                embedings = np.concatenate((embedings, pickle.load(f)), axis=0)
        
        with open(f'{stat_path}/{gender}/{natio}/labels-train.pkl','rb') as f:
            target += pickle.load(f)
    
    return (embedings, target)

In [ ]:
# Сохраняем эмбединги и метки для женщин в переменных

read_embedings_women = read_embedings(gender='women')
embedings_women = read_embedings_women[0]
target_women = read_embedings_women[1]

In [ ]:
# Проверяем соотвествие количества эмбедингов и меток для женщин

len(embedings_women), len(target_women)

In [ ]:
# Сохраняем эмбединги и метки для мужчин в переменных

read_embedings_men = read_embedings()
embedings_men = read_embedings_men[0]
target_men = read_embedings_men[1]

In [ ]:
# Проверяем соотвествие количества эмбедингов и меток для мужчин

len(embedings_men), len(target_men)

In [ ]:
# Объединяем метки мужчин и женщин пометив всех женщин 1, а всех мужчин 0. также объединяем их эмбединги. Ещё раз проверяем
# количество.

target = [1 for i in target_women] + [0 for i in target_men]
embedings = np.concatenate((embedings_women, embedings_men), axis=0)
len(embedings), len(target)

In [ ]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target, test_size=0.2, random_state=21)

In [ ]:
%%time
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
f1 = f1_score(y_test, model.predict(X_test))
print(f'F1 score = {f1}')

F-мера для обученной модели на определение пола приближается к 100%. Поэтому, считаем задача определения пола решена. Сохраняем обученную модель для последующего использования в продакшине.

In [ ]:
pkl_patch = './Documents/DataScience/PET-project/static/models'

if os.path.exists(pkl_patch) != True:
    os.mkdir(pkl_patch)

with open(f'{pkl_patch}/model_gender_determination.pkl','wb') as f:
    pickle.dump(model, f)

Далее обучим модели на поиск лиц по имеющемуся в папке **target** датасету.

### 3. Обучение моделей на распознавание лиц

Согласно поставленной задаче необходимо обучить четыре модели:
* иностранных актёров;
* отечественных актёров;
* иностранных актрис;
* отечественных актрис.

Обученные модели сохранить для последующего использования в продакшине.

#### Модель для иностранных актёров

In [ ]:
# Подгрузим сохранённые эмбединги иностранных актёров для считывания

pkl_patch = './Documents/DataScience/PET-project/static/men/eng'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [ ]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

In [ ]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [ ]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [ ]:
len(target_)

In [ ]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.25, stratify=target_, random_state=21)

Линейную регрессию будем использовать с решающим алгоритмом *solver='liblinear'* и коэффициентом регуляризации *C=50*.

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

In [ ]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

In [ ]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/eng_actors.pkl','wb') as f:
    pickle.dump(model, f)"""

#### Модель для отечественных актёров

In [ ]:
# Подгрузим сохранённые эмбединги отечественных актёров для считывания

pkl_patch = './Documents/DataScience/PET-project/static/men/rus'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [ ]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

In [ ]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [ ]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [ ]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.275, stratify=target_, random_state=21)

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

In [ ]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

In [ ]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/rus_actors.pkl','wb') as f:
    pickle.dump(model, f)"""

#### Модель для иностранных актрис

In [ ]:
# Подгрузим сохранённые эмбединги иностранных актрис для считывания

pkl_patch = './Documents/DataScience/PET-project/static/women/eng'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [ ]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

In [ ]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [ ]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [ ]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.25, stratify=target_, random_state=21)

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

In [ ]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

In [ ]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/eng_actresses.pkl','wb') as f:
    pickle.dump(model, f)"""

#### Модель для отечественных актрис

In [ ]:
# Подгрузим сохранённые эмбединги отечественных актрис для считывания

pkl_patch = './Documents/DataScience/PET-project/static/women/rus'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [ ]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

In [ ]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [ ]:
less_one

In [ ]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [ ]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.275, stratify=target_, random_state=21)

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

In [ ]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

In [ ]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/rus_actresses.pkl','wb') as f:
    pickle.dump(model, f)"""

После обучения F-мера для модели иностранных актёров составила 96,19%, для отечественных актёров - 96,56%, для иностранных актрис - 96,42%, для отечественных актрис - 97,17%. Будем использовать эти модели в продакшине.